In [1]:
from rdflib.plugins.stores import sparqlstore
import rdflib
from rdflib import URIRef, Literal, Graph, Dataset

import serialization
import discourse

import html
import loader
import discourse
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from IPython.core.display import HTML

import uuid

from itertools import chain

def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()

    display(Image(png))

def t2rdflibg(triples): #triples to rdflib graph
    g = Graph()
    for t in triples:
        g.add(t)
    return g


def nan2None(value):
    if pd.isnull(value) :
        return None
    else:
        return value


def get_data_rows(filename):
    data_rows_df = pd.read_csv(filename)
    data_rows=[]
    for i,r in data_rows_df.iterrows():
        r_d = {k:nan2None(v) for k,v in dict(r).items()}

        data_rows.append(r_d)
    del data_rows_df
    return data_rows


jena = sparqlstore.SPARQLUpdateStore("http://localhost:3030/modelg/query",context_aware=True)
#jena.open(("http://localhost:3030/modelg/query", "http://localhost:3030/modelg/update"))
jena.open(("http://localhost:3030/models/query", "http://localhost:3030/models/update"))

# Create a Dataset
ds = Dataset(store=jena, default_union=True, default_graph_base="http://base.raw")
#ds = Dataset(store=jena, default_union=True)
# Define a named graph within the dataset, this will contain all nodes contained within the graph referenced, or none if it doesn't exist yet.
sg = ds.graph(URIRef("http://config"))
sg = Graph()
master_g = ds.graph(URIRef("http://master"))
discourse_g = ds.graph(URIRef("http://discourse"))



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MappingMetaTarget belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, owl.topObjectProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit:/usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rd/q3bhk6_n56x75m05pp2sq7dr0000gn/T/tmpk6l2w0r9


Serialization.Meta [Serialization.SerializationMetaData]
Serialization.Mapping [Serialization.SerializationSpecs]
Serialization.BatchDefinition [Serialization.SerializationSpecs]
Serialization.SerializationSpecs [Serialization.SerializationMetaData]
Serialization.BatchNode [Serialization.DataLoad]
Serialization.DataLoad [Serialization.SerializationMetaData]
Serialization.SerializationMetaData [owl.Thing]
Serialization.DataPropertyMapping [Serialization.Mapping]
Serialization.MetaDataProperty [Serialization.Meta]
Serialization.EntityMapping [Serialization.Mapping]
Serialization.MetaClass [Serialization.Meta]
Serialization.JobNode [Serialization.DataLoad]
Serialization.MetaProperty [Serialization.Meta]
Serialization.PropertyMapping [Serialization.Mapping]
Serialization.RowNode [Serialization.DataLoad]
Serialization.Serialization [Serialization.SerializationSpecs]


* Owlready2 * HermiT took 0.5391039848327637 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [18]:
def meta_data_package_template(field_d):
    dc_terms_base = "http://purl.org/dc/terms/"
    rdf_form = {}
    for k,v in field_d.items():
        rdf_form[URIRef(dc_terms_base + k)] = Literal(v)
    return rdf_form


def triples_to_quads(triples, graph="http://master"):
    for s,p,o, *_ in triples:
        yield (s,p,o,URIRef(graph))

In [16]:
if len(sg)==0:
    sg.parse("DMEAR_ser.rdf")
S = serialization.Serialization(sg, "EntityAttributeRecord")

In [13]:
model_data = pd.read_csv(r"sample/inventory_v1.csv", index_col="Sequence").fillna("")
d_p = meta_data_package_template( {"created" : "2023-04-10", 
                                   "creator" : "tomk", 
                                   "description" : "Sample ERD model version 1", 
                                   "title" : "inventory", 
                                   "modified" : "2023-04-10"})

rows = [dict({rk:html.escape(str(rv)) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

In [19]:
# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])
p,de,di = loader.generate_discourse("Inventory ERD v1.0", mq, d_p)

ds.addN(triples_to_quads(mq, URIRef("http://master")))
ds.addN(triples_to_quads(p, URIRef("http://discourse")))
ds.addN(triples_to_quads(de, URIRef("http://discourse")))
ds.addN(triples_to_quads(di, URIRef("http://discourse")))


0:00:00.146289 for 9


In [20]:
model_data = pd.read_csv(r"sample/inventory_v2.csv", index_col="Sequence").fillna("")
d_p = meta_data_package_template( {"created" : "2023-04-11", 
                                   "creator" : "tomk", 
                                   "description" : "Sample ERD model version 2", 
                                   "title" : "inventory ERD", 
                                   "modified" : "2023-04-11"})

rows = [dict({rk:html.escape(str(rv)) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])
p,de,di = loader.generate_discourse("Inventory ERD v2.0", mq, d_p)

ds.addN(triples_to_quads(mq, URIRef("http://master")))
ds.addN(triples_to_quads(p, URIRef("http://discourse")))
ds.addN(triples_to_quads(de, URIRef("http://discourse")))
ds.addN(triples_to_quads(di, URIRef("http://discourse")))


0:00:00.197643 for 18


In [21]:
model_data = pd.read_csv(r"sample/inventory_v3.csv", index_col="Sequence").fillna("")
d_p = meta_data_package_template( {"created" : "2023-04-14", 
                                   "creator" : "tomk", 
                                   "description" : "Sample ERD model version 3", 
                                   "title" : "inventory ERD", 
                                   "modified" : "2023-04-14"})

rows = [dict({rk:html.escape(str(rv)) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

# Here master_g is a master graph sourced from Fuseki
mq,me_m = loader.get_triples(sg, "EntityAttributeRecord", rows, master_g, [])
p,de,di = loader.generate_discourse("Inventory ERD v3.0", mq, d_p)

ds.addN(triples_to_quads(mq, URIRef("http://master")))
ds.addN(triples_to_quads(p, URIRef("http://discourse")))
ds.addN(triples_to_quads(de, URIRef("http://discourse")))
ds.addN(triples_to_quads(di, URIRef("http://discourse")))


0:00:00.303726 for 44


In [22]:
# Step 1 - Tell me what discourses are available in the system

In [35]:
discourse_details_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX disco: <http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

select ?g ?discourse ?name ?title ?description ?created (COUNT(?declaration) as ?declarations) (COUNT(distinct ?psubject) as ?entities)

WHERE {
  GRAPH ?g
  {
      ?discourse a disco:Discourse.
      ?discourse dcterms:title ?title.
      ?discourse rdfs:label ?name.
      ?discourse dcterms:description ?description.
      ?discourse dcterms:created ?created .
      ?discourse disco:DiscourseContains+ ?declaration .
      ?declaration a disco:Declaration .
      ?posit_type rdfs:subPropertyOf* disco:Posits .
      ?declaration disco:Asserts ?assertion.
      ?assertion disco:Subject ?psubject.
  }
}
GROUP BY ?g ?discourse ?name ?title ?description ?created 
"""

qr = ds.query(discourse_details_sparql)

In [37]:
pd.DataFrame(qr, columns=[v.n3()[1:] for v in qr.vars]).sort_values(by="created")

,g,discourse,name,title,description,created,declarations,entities
1,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_test_123,some title,test data package,2023-04-01,13775,2534
2,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_test_123,some title,test data package,2023-04-01,13775,2534
3,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_test_123,some title,test data package,2023-04-01,13775,2534
6,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_test_123,some title,test data package,2023-04-01,13775,2534
7,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_test_123,some title,test data package,2023-04-01,13775,2534
0,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_Inventory ERD v1.0,inventory,Sample ERD model version 1,2023-04-10,97,16
5,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_Inventory ERD v2.0,inventory ERD,Sample ERD model version 2,2023-04-11,168,29
4,http://discourse,http://www.semanticweb.org/tomk/ontologies/202...,discourse_Inventory ERD v3.0,inventory ERD,Sample ERD model version 3,2023-04-14,337,61


https://docs.data.world/tutorials/sparql/list-of-sparql-filter-functions.html

In [62]:
get_posits_by_discourse_sparql = """

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX disco: <http://www.semanticweb.org/tomk/ontologies/2022/11/discourse#> 
PREFIX dcterms: <http://purl.org/dc/terms/> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

select ?g ?discourse ?assertion ?refutation ?posit
WHERE {
  GRAPH ?g
  {
      ?discourse a disco:Discourse.
      ?discourse disco:DiscourseContains+ ?declaration .
      ?declaration a disco:Declaration .
        OPTIONAL { ?declaration disco:Asserts ?assertion. }
        OPTIONAL { ?declaration disco:Refutes ?refutation. }
        OPTIONAL { ?declaration disco:Posits ?posit. }
      FILTER (?discourse IN ( <http://www.semanticweb.org/tomk/ontologies/2022/11/ed34219418a041638da83b9fc66b1755>, <http://www.semanticweb.org/tomk/ontologies/2022/11/20efd3b4ce8d4738bb483cb4fb50d708> )) .
  }
}

"""

qr = ds.query(get_posits_by_discourse_sparql)

In [63]:
discourse_contents = pd.DataFrame(qr, columns=[v.n3()[1:] for v in qr.vars]).sort_values(by="discourse")

In [64]:
d_set_d = discourse_contents.groupby(['discourse'])['assertion'].agg(set).to_dict()

In [65]:
key_l = list(d_set_d.keys())

In [66]:
len(d_set_d[key_l[0]].intersection(d_set_d[key_l[1]])), len(d_set_d[key_l[0]].difference(d_set_d[key_l[1]])), len(d_set_d[key_l[1]].difference(d_set_d[key_l[0]]))




(97, 240, 0)